# Default Rate Sensitivity on Marketing Spend Framework - MX

The effects of changing default rates on Liang & Zu's marketing spend framework is explored.

---

In [17]:
import pandas as pd
import numpy as np
import pyltv
from dbm import DBM
import plotly
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit

In [18]:
data = pd.read_csv('data/mx_data.csv')

### Study Assumptions
1. Default rate stress applied as a blanket stress across all time periods.
2. Default rate stress applied to both 7dpd and 365dpd default rates.
3. The default rate stress is simply a percentage point added to the actual default rate. **So if the 365dpd default rate is currently 6%, adding a 1% stress results in a new 365dpd default rate of 7%**.

## Impact of Default Rate on LTV

In [19]:
min_months=4
n_months=50
default_stress=0.01

m1 = pyltv.Model(data, market='mx', fcast_method='powerslope')
m1.generate_features()
m1.forecast = m1.forecast_data(m1.data, min_months, n_months)

m2 = pyltv.Model(data, market='mx', fcast_method='powerslope', default_stress=default_stress)
m2.generate_features()
m2.forecast = m2.forecast_data(m2.data, min_months, n_months)

Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...
Data spans 2020-09 to 2022-01
Total # of cohorts: 17
...


In [99]:
colors = {i: c for i, c in enumerate(plotly.colors.qualitative.Dark24)}
# add additional colors
color_size = len(colors)
for i, c in enumerate(plotly.colors.qualitative.Light24):
    colors[i+color_size] = c

fig = make_subplots(rows=1, cols=2, shared_xaxes=True,
                   subplot_titles=('Unstressed', '+1% Default Rates'))

param = 'cumulative_dcf_ltv_per_original'

# LEFT PLOT
curves1=[]
for cohort in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort == cohort]
    for dtype in c_data.data_type.unique():
        output = c_data[c_data.data_type == dtype][param]

        output.name = cohort + '-' + dtype

        curves1.append(output)
    
for i, cohort in enumerate(curves1):
    c = colors[i]
    if 'forecast' in cohort.name:
        fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='lines',
                                 line=dict(width=3, dash='dash', color=c), legendgroup=f'{i}'), row=1, col=1)
    else:
        if cohort.notnull().any():
            fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='markers+lines',
                                     line=dict(width=2, color=c), legendgroup=f'{i}'), row=1, col=1)

expected = m1.ltv_expected.cumulative_ltv_per_original

fig.add_trace(go.Scatter(name='expected', x=expected.index, y=expected, 
                         line=dict(color='black', width=3), legendgroup='e'), row=1, col=1)
fig.add_trace(go.Scatter(name='expected', x=expected.index, y=expected, 
                        line=dict(color='black', width=3), legendgroup='e', showlegend=False), row=1, col=2)

# RIGHT PLOT
curves2=[]
for cohort in m2.forecast.cohort.unique():
    c_data = m2.forecast[m2.forecast.cohort == cohort]
    for dtype in c_data.data_type.unique():
        output = c_data[c_data.data_type == dtype][param]

        output.name = cohort + '-' + dtype

        curves2.append(output)
        
for i, cohort in enumerate(curves2):
    c = colors[i]
    if 'forecast' in cohort.name:
        fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='lines',
                                 line=dict(width=3, dash='dash', color=c), legendgroup=f'{i}', showlegend=False), row=1, col=2,)
    else:
        if cohort.notnull().any():
            fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='markers+lines',
                                     line=dict(width=2, color=c), legendgroup=f'{i}', showlegend=False), row=1, col=2)

fig.update_layout(title='DCF LTV')
fig.layout.xaxis.title='Month'
fig.layout.xaxis2.title='Month'
fig.update_yaxes(range=[-10, 85])
fig.update_xaxes(range=[-5,125])

fig.show()

In [67]:
m1.plot_cohorts('default_rate_365dpd', data='forecast')
m2.plot_cohorts('default_rate_365dpd', data='forecast')

### Standard Default Rate

In [68]:
data_50mo1 = m1.forecast[m1.forecast['Months Since First Loan Disbursed']==50]
ltv1 = data_50mo1['cumulative_ltv_per_original'].median()
dcf_ltv1 = data_50mo1['cumulative_dcf_ltv_per_original'].median()
#print(f'50mo LTV: ${round(ltv1, 2)}')
print(f'Median DCF 50mo LTV: ${round(dcf_ltv1, 2)}')

Median DCF 50mo LTV: $58.2


### +1% Default Rate Stress

In [69]:
data_50mo2 = m2.forecast[m2.forecast['Months Since First Loan Disbursed']==50]
ltv2 = data_50mo2['cumulative_ltv_per_original'].median()
dcf_ltv2 = data_50mo2['cumulative_dcf_ltv_per_original'].median()
#print(f'50mo LTV: ${round(ltv2, 2)}')
print(f'Median DCF 50mo LTV: ${round(dcf_ltv2, 2)}')

Median DCF 50mo LTV: $49.49


In [70]:
print('Change in DCF LTV:')
print(f'{round(100*(dcf_ltv2-dcf_ltv1)/dcf_ltv1, 2)}%')

Change in DCF LTV:
-14.96%


In [71]:
ltvs = {}
for x in np.linspace(-0.03, 0.03, 7):
    m = pyltv.Model(data, dollar_ex=51, market='ph', fcast_method='powerslope', default_stress=x)
    m.generate_features()
    m.forecast = m.forecast_data(m.data, min_months, n_months)
    
    data_50mo = m.forecast[m.forecast['Months Since First Loan Disbursed']==50]
    m.forecast['defaulted_origination'] = m.forecast['default_rate_365dpd']*m.forecast.origination_per_original
    default = m.forecast.defaulted_origination.sum()/m.forecast.origination_per_original.sum()
    
    ltvs[round(x, 2)] = (round(data_50mo['cumulative_dcf_ltv_per_original'].median(), 2), 
                         round(100*default, 2))

Data spans 2020-09 to 2021-12
Total # of cohorts: 16
...
Data spans 2020-09 to 2021-12
Total # of cohorts: 16
...
Data spans 2020-09 to 2021-12
Total # of cohorts: 16
...
Data spans 2020-09 to 2021-12
Total # of cohorts: 16
...
Data spans 2020-09 to 2021-12
Total # of cohorts: 16
...
Data spans 2020-09 to 2021-12
Total # of cohorts: 16
...
Data spans 2020-09 to 2021-12
Total # of cohorts: 16
...


### Default Rate Stress: -3% to +3%

The plot below shows the relationship between a change in the default rate stress and the median DCF LTV for all cohorts.

In [72]:
v = [i[0] for i in ltvs.values()]
l = [f'{i[1]}%' for i in ltvs.values()]
fig = go.Figure(go.Scatter(x=l, y=v))
fig.update_layout(xaxis=dict(title='Default Rate Stress'),
                 yaxis=dict(title='Median DCF LTV'))
fig.show()

In [73]:
## CHANGE TO WEIGHTED AVERAGE 

In [74]:
print('For every +1% increase in default rate...')
print(f'-${round(ltvs[0][0]-ltvs[0.01][0], 2)}')

For every +1% increase in default rate...
-$8.71


The relationships between LTV and default rates is shown below and explains the linear trend. **Note that this analysis does not include the effects of default rate changes on retention.**

LTV ~ [revenue - (origination+revenue)*default_rate_365dpd]

revenue ~ [0.08 * default_rate_7dpd]

## Default Rate Historicals

#### Weighted Average Lifetime Default

This method uses the average lifetime default rate (365dpd) for each cohort. The average lifetime default rate is a weighted average of defaults by origination_per_original.

In [75]:
defaults = []
default_changes = []
avg_lifetime_default = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c][['origination_per_original', 'default_rate_365dpd']]
    c_data['defaulted_origination'] = c_data['default_rate_365dpd']*c_data.origination_per_original
    c_data['defaulted_origination_change'] = (c_data.defaulted_origination-c_data.defaulted_origination.shift(1))/ \
        c_data.defaulted_origination
    
    defaults.append(go.Scatter(name=c, x=c_data.index, y=c_data.defaulted_origination))
    default_changes.append(go.Scatter(name=c, x=c_data.index, y=c_data.defaulted_origination_change))
    
    avg_lifetime_default.append(c_data.defaulted_origination.sum()/c_data.origination_per_original.sum())
    
avg_lifetime_default = pd.Series(avg_lifetime_default, index=m1.forecast.cohort.unique())

In [76]:
fig=go.Figure(defaults)
fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Defaulted Origination per Original'))
fig.show()

In [77]:
fig = go.Figure(
    go.Scatter(name=c, x=avg_lifetime_default.index, y=avg_lifetime_default, mode='markers',
                              marker=dict(size=15))
)

fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Default Rate'))
fig.show()

In PH, we don't see the same trend of increasing defaults with time. The highest weighted average default rate we see in any cohort is 6.82%. 

#### Unweighted Average Default

In this method, we simply look at aggregate stats of the 365 dpd default rate, without taking into account origination. This scenario looks at a blanket default rate increase across the board, regardless of the size of loans.

In [78]:
default_stats = {}
for i, c in enumerate(m1.forecast.cohort.unique()):
    c_data = m1.forecast[m1.forecast.cohort==c][['default_rate_365dpd']]
    
    default_stats[c] = (float(c_data.median()), float(c_data.quantile(0.9)))

In [79]:
medians=[]
quantile90=[]
for c in default_stats:
    medians.append(default_stats[c][0])
    quantile90.append(default_stats[c][1])
    
medians = pd.Series(medians, index=np.arange(1, len(medians)+1))
#params1, covs1 = curve_fit(reg, xdata=medians.index, ydata=medians)

quantile90 = pd.Series(quantile90, index=np.arange(1, len(medians)+1))
#params2, covs2 = curve_fit(reg, xdata=quantile90.index, ydata=quantile90)
    
fig = go.Figure([
    go.Scatter(name='median default', x=medians.index, y=medians, mode='markers'),
    #go.Scatter(name='median-fit', x=np.arange(1,36), y=reg(np.arange(1,36), params1[0], params1[1])),
    go.Scatter(name='90 percentile', x=quantile90.index, y=quantile90, mode='markers')
    #go.Scatter(name='90 percentile-fit', x=np.arange(1,36), y=reg(np.arange(1,36), params2[0], params2[1]))
])

fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Default Rate'))
fig.show()

Similarly, in looking at both the 50% and 90% quantiles of the 365dpd default rates, the trend is quite flat. Here, we see a **maximum of 7.9%**.

## Payback Period

### mCAC Payback Period

In [80]:
ratios = [1, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        if r == 2:
            cac_ratio = 4.41
        elif r == 1.5:
            cac_ratio = 3.73
        elif r == 1:
            cac_ratio = 2.73
            
        avg_cac = float(dcf_ltv/cac_ratio)

        dcf = c_data['cumulative_dcf_ltv_per_original']
        payback_idx = dcf[dcf >= mcac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results1 = pd.DataFrame.from_records(paybacks)

In [81]:
ratios = [1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    c_data_stressed = m2.forecast[m2.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    dcf_ltv_stressed = float(c_data_stressed[c_data_stressed['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv, 'DCF LTV Stressed': dcf_ltv_stressed}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        if r == 2:
            cac_ratio = 4.41
        elif r == 1.5:
            cac_ratio = 3.73
        elif r == 1:
            cac_ratio = 2.73
            
        avg_cac = float(dcf_ltv/cac_ratio)
        
        dcf = c_data_stressed['cumulative_dcf_ltv_per_original']
        
        payback_idx = dcf[dcf >= mcac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results2 = pd.DataFrame.from_records(paybacks)

The table below shows the payback periods for unstressed default rates in the 1.5x scenario.

In [82]:
ratio = 1.5
payback_results1[['cohort', 'DCF LTV', f'mCAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

,cohort,DCF LTV,mCAC (1.5),Payback Period (1.5),Payback Amount (1.5)
0,2020-09,55.455498,36.970332,26,37.694635
1,2020-10,51.459776,34.306518,26,34.818683
2,2020-11,46.797354,31.198236,26,31.507524
3,2020-12,57.087412,38.058275,25,38.213294
4,2021-01,46.103693,30.735796,27,31.833471
5,2021-02,56.099652,37.399768,26,38.178907
6,2021-03,71.291832,47.527888,25,48.230865
7,2021-04,71.623252,47.748835,25,48.333149
8,2021-05,68.021518,45.347678,25,45.697407
9,2021-06,64.732749,43.155166,25,43.386228


The second table shows the new payback periods in the same 1.5x scenario, but with a +1% default rate stress applied. The payback periods are pushed out by several months.

In [83]:
ratio = 1.5
payback_results2[['cohort', 'DCF LTV', 'DCF LTV Stressed', f'mCAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

,cohort,DCF LTV,DCF LTV Stressed,mCAC (1.5),Payback Period (1.5),Payback Amount (1.5)
0,2020-09,55.455498,47.483787,36.970332,32,37.548146
1,2020-10,51.459776,43.743562,34.306518,32,34.420753
2,2020-11,46.797354,39.295766,31.198236,33,31.495671
3,2020-12,57.087412,48.191682,38.058275,32,38.543471
4,2021-01,46.103693,38.696156,30.735796,33,30.828734
5,2021-02,56.099652,47.933965,37.399768,32,37.912464
6,2021-03,71.291832,61.800698,47.527888,30,47.625954
7,2021-04,71.623252,61.904527,47.748835,31,48.748127
8,2021-05,68.021518,58.695566,45.347678,31,46.068040
9,2021-06,64.732749,55.625431,43.155166,31,43.578986


In [84]:
fig1 = go.Figure([go.Scatter(name='1x', x=payback_results1['Payback Period (1)'], 
                            y=payback_results1['Payback Amount (1)'], mode='markers',
                            marker=dict(color='red')),
                  go.Scatter(name='1.5x', x=payback_results1['Payback Period (1.5)'], 
                            y=payback_results1['Payback Amount (1.5)'], mode='markers',
                            marker=dict(color='blue')),
                 go.Scatter(name='2x', x=payback_results1['Payback Period (2)'], 
                            y=payback_results1['Payback Amount (2)'], mode='markers',
                            marker=dict(color='orange'))
                ])

fig1.update_layout(title='Unstressed Default Rate')
fig1.update_layout(xaxis=dict(title='Month', range=[18, 35]), yaxis=dict(title='DCF LTV'))

fig2 = go.Figure([go.Scatter(name='1.5x', x=payback_results2['Payback Period (1.5)'], 
                            y=payback_results2['Payback Amount (1.5)'], mode='markers',
                            marker=dict(color='blue')),
                 go.Scatter(name='2x', x=payback_results2['Payback Period (2)'], 
                            y=payback_results2['Payback Amount (2)'], mode='markers',
                            marker=dict(color='orange'))
                ])

fig2.update_layout(title='+1% Default Rate')
fig2.update_layout(xaxis=dict(title='Month', range=[18, 35]), yaxis=dict(title='DCF LTV'))

fig1.show()
fig2.show()

In [85]:
payback_results1['Payback Period (2)'] - payback_results2['Payback Period (2)']

0    -4
1    -4
2    -4
3    -3
4    -4
5    -4
6    -3
7    -3
8    -4
9    -4
10   -4
11   -3
Name: Payback Period (2), dtype: int64

In the worst case, our most recent cohort, the **payback period will shift out 7 months at an LTV/mCAC of 1.5**, and by **4 months for an LTV/mCAC of 2**.

### Average CAC Payback Period

In [86]:
ratios = [1, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        if r == 2:
            cac_ratio = 4.41
        elif r == 1.5:
            cac_ratio = 3.73
        elif r == 1:
            cac_ratio = 2.73
            
        avg_cac = float(dcf_ltv/cac_ratio)

        dcf = c_data['cumulative_dcf_ltv_per_original']
        payback_idx = dcf[dcf >= avg_cac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results1 = pd.DataFrame.from_records(paybacks)

In [87]:
ratios = [1, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    c_data_stressed = m2.forecast[m2.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    dcf_ltv_stressed = float(c_data_stressed[c_data_stressed['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv, 'DCF LTV Stressed': dcf_ltv_stressed}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        if r == 2:
            cac_ratio = 4.41
        elif r == 1.5:
            cac_ratio = 3.73
        elif r == 1:
            cac_ratio = 2.73
            
        avg_cac = float(dcf_ltv/cac_ratio)
        
        dcf = c_data_stressed['cumulative_dcf_ltv_per_original']
        
        payback_idx = dcf[dcf >= avg_cac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results2 = pd.DataFrame.from_records(paybacks)

The two tables below show the changes in payback periods when using LTV/CAC instead of LTV/mCAC as the guard rail. The shifts in payback periods are much less significant.

In [88]:
ratio = 1.5
payback_results1[['cohort', 'DCF LTV', f'CAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

,cohort,DCF LTV,CAC (1.5),Payback Period (1.5),Payback Amount (1.5)
0,2020-09,55.455498,14.867426,14,15.487649
1,2020-10,51.459776,13.796187,14,14.323908
2,2020-11,46.797354,12.546207,14,12.809775
3,2020-12,57.087412,15.304936,13,15.577505
4,2021-01,46.103693,12.360240,15,13.943268
5,2021-02,56.099652,15.040121,14,16.212317
6,2021-03,71.291832,19.113092,13,20.563840
7,2021-04,71.623252,19.201944,13,20.459628
8,2021-05,68.021518,18.236332,13,19.027983
9,2021-06,64.732749,17.354624,13,17.877410


In [89]:
ratio = 1.5
payback_results2[['cohort', 'DCF LTV', 'DCF LTV Stressed', f'CAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

,cohort,DCF LTV,DCF LTV Stressed,CAC (1.5),Payback Period (1.5),Payback Amount (1.5)
0,2020-09,55.455498,47.483787,14.867426,16,15.709551
1,2020-10,51.459776,43.743562,13.796187,16,14.046701
2,2020-11,46.797354,39.295766,12.546207,17,13.950037
3,2020-12,57.087412,48.191682,15.304936,15,15.418897
4,2021-01,46.103693,38.696156,12.360240,17,13.174383
5,2021-02,56.099652,47.933965,15.040121,16,16.114038
6,2021-03,71.291832,61.800698,19.113092,15,21.101291
7,2021-04,71.623252,61.904527,19.201944,15,20.934051
8,2021-05,68.021518,58.695566,18.236332,15,19.419340
9,2021-06,64.732749,55.625431,17.354624,15,18.149514


In [90]:
fig1 = go.Figure([go.Scatter(name='1x', x=payback_results1['Payback Period (1)'], 
                            y=payback_results1['Payback Amount (1)'], mode='markers'),
                  go.Scatter(name='1.5x', x=payback_results1['Payback Period (1.5)'], 
                            y=payback_results1['Payback Amount (1.5)'], mode='markers'),
                 go.Scatter(name='2x', x=payback_results1['Payback Period (2)'], 
                            y=payback_results1['Payback Amount (2)'], mode='markers')
                ])

fig1.update_layout(title='Unstressed Default Rate')
fig1.update_layout(xaxis=dict(title='Month', range=[8, 22]), yaxis=dict(title='DCF LTV'))

fig2 = go.Figure([go.Scatter(name='1x', x=payback_results2['Payback Period (1)'], 
                            y=payback_results2['Payback Amount (1)'], mode='markers'),
                  go.Scatter(name='1.5x', x=payback_results2['Payback Period (1.5)'], 
                            y=payback_results2['Payback Amount (1.5)'], mode='markers'),
                 go.Scatter(name='2x', x=payback_results2['Payback Period (2)'], 
                            y=payback_results2['Payback Amount (2)'], mode='markers')
                ])

fig2.update_layout(title='+1% Default Rate')
fig2.update_layout(xaxis=dict(title='Month', range=[8, 22]), yaxis=dict(title='DCF LTV'))

fig1.show()
fig2.show()

In [91]:
payback_results1['Payback Period (1.5)'] - payback_results2['Payback Period (1.5)']

0    -2
1    -2
2    -3
3    -2
4    -2
5    -2
6    -2
7    -2
8    -2
9    -2
10   -2
11   -2
Name: Payback Period (1.5), dtype: int64

In the worst case, our most recent cohort, the **payback period will shift out by 2 months at an LTV/CAC of 1.5x**, and by **2 for an LTV/CAC of 2x**.

## Conclusions

- A rise in default rate leads to an increas in payback period. This is true for any scenario (1x, 1.5x, 2x). In general, the shift in payback period is worse for lower multipliers (less margin).
- A +1% increase in default rate in the 1.5x scenario will shift the payback period by up to 7 months. At 2x, there is only a shift in payback period of +2 months.
- With a 1.5x scenario, the data suggests we can still maintain a 50mo positive LTV inspite of up to a +1% rise in default rates.